In [1]:
import pandas as pd
import numpy as np
import math
import re

In [2]:
#Load original taxonomy
vse = pd.read_csv('VSE Research Portfolio.csv')

In [3]:
#Split Category by certian characters then expand dataframe using explode
new = vse['Category'].str.split("; and")
vse['Category'] = new
vse = vse.explode('Category')
new = vse['Category'].str.split(";")
vse['Category'] = new
vse = vse.explode('Category')
new = vse['Category'].str.split("/")
vse['Category'] = new
vse = vse.explode('Category')
vse['Category'] = vse['Category'].str.strip()
vse

,Topic,Area,Category
0,Digital Systems and Data,Artificial Intelligence,Cognitive Systems
0,Digital Systems and Data,Artificial Intelligence,Intelligent Agents
1,Digital Systems and Data,Artificial Intelligence,Knowledge Representation and Reasoning
2,Digital Systems and Data,Artificial Intelligence,Machine Learning
3,Digital Systems and Data,Artificial Intelligence,Natural Language Processing
...,...,...,...
130,Specific Business Sectors,Transportation,Safety and Security
130,Specific Business Sectors,Transportation,Emergency Response
131,Specific Business Sectors,Transportation,Sustainability and Resilience
132,Specific Business Sectors,Transportation,System Management


In [4]:
#Split Area by certian characters then expand dataframe using explode
new = vse['Area'].str.split("; and")
vse['Area'] = new
vse = vse.explode('Area')
new = vse['Area'].str.split(";")
vse['Area'] = new
vse = vse.explode('Area')
vse['Area'] = vse['Area'].str.strip()
vse

,Topic,Area,Category
0,Digital Systems and Data,Artificial Intelligence,Cognitive Systems
0,Digital Systems and Data,Artificial Intelligence,Intelligent Agents
1,Digital Systems and Data,Artificial Intelligence,Knowledge Representation and Reasoning
2,Digital Systems and Data,Artificial Intelligence,Machine Learning
3,Digital Systems and Data,Artificial Intelligence,Natural Language Processing
...,...,...,...
130,Specific Business Sectors,Transportation,Safety and Security
130,Specific Business Sectors,Transportation,Emergency Response
131,Specific Business Sectors,Transportation,Sustainability and Resilience
132,Specific Business Sectors,Transportation,System Management


In [5]:
#Set unique ID values for each Topic, Area, and Category
from collections import defaultdict

temp = defaultdict(lambda: len(temp))
Topic_ID = [temp[ele] for ele in vse['Topic']]
Topic_ID = [str(x) for x in Topic_ID]
Topic_ID = ["t" + s for s in Topic_ID]
temp = defaultdict(lambda: len(temp))
Area_ID = [temp[ele] for ele in vse['Area']]
Area_ID = [str(x) for x in Area_ID]
Area_ID = ["a" + s for s in Area_ID]
temp = defaultdict(lambda: len(temp))
Category_ID = [temp[ele] for ele in vse['Category']]
Category_ID = [str(x) for x in Category_ID]
Category_ID = ["c" + s for s in Category_ID]

vse['Topic_ID'] = Topic_ID
vse['Area_ID'] = Area_ID
vse['Category_ID'] = Category_ID
#Adjust Capitalization
vse['Topic'] = vse['Topic'].str.lower()
vse['Area'] = vse['Area'].str.lower()
vse['Category'] = vse['Category'].str.lower()
vse

,Topic,Area,Category,Topic_ID,Area_ID,Category_ID
0,digital systems and data,artificial intelligence,cognitive systems,t0,a0,c0
0,digital systems and data,artificial intelligence,intelligent agents,t0,a0,c1
1,digital systems and data,artificial intelligence,knowledge representation and reasoning,t0,a0,c2
2,digital systems and data,artificial intelligence,machine learning,t0,a0,c3
3,digital systems and data,artificial intelligence,natural language processing,t0,a0,c4
...,...,...,...,...,...,...
130,specific business sectors,transportation,safety and security,t5,a43,c158
130,specific business sectors,transportation,emergency response,t5,a43,c159
131,specific business sectors,transportation,sustainability and resilience,t5,a43,c160
132,specific business sectors,transportation,system management,t5,a43,c161


In [6]:
#Save expanded dictionary
vse.to_csv('VSE Research Portfolio Explode.csv')


In [10]:
#Load other files to obtain other stated research interests
CEC = pd.read_csv('loadable_profiles_final_all.csv')
GS = pd.read_csv('Scholar_GMU_Names.csv')
GS_pubs = pd.read_csv('GoogleScholar Final Dataset.csv')

In [11]:
#Adjust names and take neccessary columns
CEC['names'] = CEC['Name']
CEC_temp = CEC[['ID', 'names', 'Research Interest']]

In [12]:
#Search for names in CEC names that aren't in the Google Scholar names
unique_names = GS_pubs['authors'].unique()
GS = GS[['names', 'topics']]
GS
common_names = GS.loc[GS['names'].isin(unique_names)]
df = pd.DataFrame(unique_names, columns = ['names'])
missing_names = df.loc[~df['names'].isin(common_names['names'])]
missing_names

,names
3,NaN
5,Tokunbo Fadahunsi
8,Alexander H Levis
10,Ali K. Raz
12,Antonios Anastasopoulos
19,Brittany Johnson-Matthews
24,Celso Moller Ferreira
29,Dimitrios Ioannou
61,Jonathan L. Auerbach
74,Krzysztof Gaj


In [27]:
#Create Dataframe with research interests from Google Scholar topics and CEC stated research interests
common = pd.merge(common_names, CEC_temp, on = 'names')
missing = pd.merge(missing_names, CEC_temp, on = 'names')
common_and_missing = common.append(missing, ignore_index = True)
common_and_missing = common_and_missing.sort_values(by=['ID'])
common_and_missing = common_and_missing.rename(columns={'topics': 'Google Scholar topics'})
common_and_missing = common_and_missing.reset_index()
common_and_missing = common_and_missing.drop(columns=['index'])
common_and_missing

,names,Google Scholar topics,ID,Research Interest
0,Omoche Agada,"Cybersecurity, Digital Forensics",2,NaN
1,Pouyan Ahmadi,"IoT Security, Wireless Communications, Modelin...",3,NaN
2,Massimiliano Albanese,Cyber Security,4,"Entrepreneurship and Innovation, Information S..."
3,Jan Allbeck,Virtual Humans,5,"Artificial Intelligence, Computer Game Technol..."
4,Paul Ammann,Software Engineering,8,"Software Engineering,"
...,...,...,...,...
120,Kai Zeng,"5G wireless security, CPS/IoT security and pri...",235,"5G Wireless Security, Spectrum Sharing, CPS/Io..."
121,Qiang Zeng,Computer Security,236,NaN
122,Linghan Zhang,"Biometric, Voice Authentication, Mobile Securi...",237,NaN
123,Xiaokuan Zhang,"System Security, Privacy, Side Channels, DeFi,...",238,NaN


In [28]:
#Combine CEC research interests and google Scholar research interests, adjust capitalization and remove nan, , 
CEC_GS = pd.merge(CEC, common_and_missing, on = ['ID','names','Research Interest'])
CEC_GS = CEC_GS.drop(columns=['names'])
CEC_GS['All_Research_Interest'] = CEC_GS['Research Interest'].astype(str) + ',' + CEC_GS['Google Scholar topics']
CEC_GS['All_Research_Interest'] = CEC_GS['All_Research_Interest'].replace('nan,','', regex=True)
CEC_GS['All_Research_Interest'] = CEC_GS['All_Research_Interest'].replace(',,',',', regex=True)
#CEC_GS = CEC_GS.replace({np.nan: 'None'})
CEC_GS['All_Research_Interest'] = CEC_GS['All_Research_Interest'].str.lower()
CEC_GS['All_Research_Interest'] = CEC_GS['All_Research_Interest'].str.split(",")
#CEC_GS = CEC_GS.replace({np.nan: 'N/A'})
CEC_GS

,ID,Name,Title,Phone Number,Biography,Research Interest,Degrees,Profile Link,Email,Personal Link,Google Scholar topics,All_Research_Interest
0,2,Omoche Agada,Assistant Professor,NaN,Omoche Cheche Agada's areas of interests incl...,NaN,"PhD, Information Technology, George Mason Univ...",https://cec.gmu.edu/node/8046,oagada@gmu.edu,NaN,"Cybersecurity, Digital Forensics","[cybersecurity, digital forensics]"
1,3,Pouyan Ahmadi,Assistant Professor,703-909-7254,Pouyan Ahmadi's research interests include coo...,NaN,"PhD, Electrical and Computer Engineering, Geo...",https://cec.gmu.edu/profiles/pahmadi,pahmadi@gmu.edu,NaN,"IoT Security, Wireless Communications, Modelin...","[iot security, wireless communications, mode..."
2,4,Massimiliano Albanese,"Associate Professor, Associate Director at the...",703-993-1629,Massimiliano Albanese is an associate Profess...,"Entrepreneurship and Innovation, Information S...","PhD, Computer Science & Engineering, Universi...",https://cec.gmu.edu/profiles/malbanes,malbanes@gmu.edu,http://csis.gmu.edu/albanese/\n,Cyber Security,"[entrepreneurship and innovation, information..."
3,5,Jan Allbeck,"Associate Professor, Associate Dean for the Ho...",703-993-1110,Jan Allbeck is the Associate Dean of the Honor...,"Artificial Intelligence, Computer Game Technol...","PhD, Computer and Information Science, Univer...",https://cec.gmu.edu/profiles/jallbeck,NaN,http://cs.gmu.edu/~jallbeck/,Virtual Humans,"[artificial intelligence, computer game techn..."
4,8,Paul Ammann,Associate Professor\nComputer Science\nCollege...,703-993-1660,Paul Ammann has taught at George Mason Univers...,"Software Engineering,","PhD, Computer Science, University of Virginia...",https://cec.gmu.edu/profiles/pammann,pammann@gmu.edu,http://cs.gmu.edu/~pammann/\n,Software Engineering,"[software engineering, software engineering]"
...,...,...,...,...,...,...,...,...,...,...,...,...
120,235,Kai Zeng,"Associate Professor, Electrical and Computer E...",703-993-5933,Kai Zeng is an Associate Professor in the Dep...,"5G Wireless Security, Spectrum Sharing, CPS/Io...","PhD, Electrical and Computer Engineering, Wor...",https://cec.gmu.edu/profiles/kzeng2,kzeng2@gmu.edu,http://people-ece.vse.gmu.edu/~kzeng2/\n,"5G wireless security, CPS/IoT security and pri...","[5g wireless security, spectrum sharing, cps..."
121,236,Qiang Zeng,Associate Professor,NaN,Dr. Qiang Zeng is an Associate Professor in t...,NaN,"PhD, Computer Science and Engineering, Penn St...",https://cec.gmu.edu/profiles/qzeng2,zeng@gmu.edu,https://cs.gmu.edu/~zeng/\n,Computer Security,[computer security]
122,237,Linghan Zhang,"Assistant Professor, Cyber Security Engineerin...",NaN,Linghan Zhang is an Assistant Professor of th...,NaN,NaN,https://cec.gmu.edu/node/7096,lzhang37@gmu.edu,https://mason.gmu.edu/~lzhang37/\n,"Biometric, Voice Authentication, Mobile Securi...","[biometric, voice authentication, mobile sec..."
123,238,Xiaokuan Zhang,Assistant Professor,NaN,Xiaokuan Zhang's research interest include co...,NaN,"PhD, Computer Science and Engineering, Ohio St...",https://cec.gmu.edu/profiles/xzhang46,xzhang46@gmu.edu,NaN,"System Security, Privacy, Side Channels, DeFi,...","[system security, privacy, side channels, d..."


In [234]:
#Remove reparted values in All_Research_Interests
test = CEC_GS['All_Research_Interest']
test
new =[]
for i in test:
    if isinstance(i, list):
        res = [*set(i)]
        new.append(res)
    else:
        new.append(np.nan)


#Expand All research interests so each interests have its own row
CEC_GS['All_Research_Interest'] = new
#CEC_GS = CEC_GS.replace({np.nan: 'N/A'})
#temp = CEC_GS['All_Research_Interest'].str.strip()
CEC_GS = CEC_GS.explode('All_Research_Interest')
CEC_GS['All_Research_Interest'] = CEC_GS['All_Research_Interest'].str.strip()
CEC_GS = CEC_GS.replace({np.nan: 'N/A'})
CEC_GS
#temp

,ID,Name,Title,Phone Number,Biography,Research Interest,Degrees,Profile Link,Email,Personal Link,Google Scholar topics,All_Research_Interest
0,2,Omoche Agada,Assistant Professor,N/A,Omoche Cheche Agada's areas of interests incl...,N/A,PhD; Information Technology; George Mason Univ...,https://cec.gmu.edu/node/8046,oagada@gmu.edu,N/A,"Cybersecurity, Digital Forensics",cybersecurity
0,2,Omoche Agada,Assistant Professor,N/A,Omoche Cheche Agada's areas of interests incl...,N/A,PhD; Information Technology; George Mason Univ...,https://cec.gmu.edu/node/8046,oagada@gmu.edu,N/A,"Cybersecurity, Digital Forensics",digital forensics
1,3,Pouyan Ahmadi,Assistant Professor,703-909-7254,Pouyan Ahmadi's research interests include coo...,N/A,PhD; Electrical and Computer Engineering; Geo...,https://cec.gmu.edu/profiles/pahmadi,pahmadi@gmu.edu,N/A,"IoT Security, Wireless Communications, Modelin...",modeling and performance evaluation
1,3,Pouyan Ahmadi,Assistant Professor,703-909-7254,Pouyan Ahmadi's research interests include coo...,N/A,PhD; Electrical and Computer Engineering; Geo...,https://cec.gmu.edu/profiles/pahmadi,pahmadi@gmu.edu,N/A,"IoT Security, Wireless Communications, Modelin...",wireless communications
1,3,Pouyan Ahmadi,Assistant Professor,703-909-7254,Pouyan Ahmadi's research interests include coo...,N/A,PhD; Electrical and Computer Engineering; Geo...,https://cec.gmu.edu/profiles/pahmadi,pahmadi@gmu.edu,N/A,"IoT Security, Wireless Communications, Modelin...",iot security
...,...,...,...,...,...,...,...,...,...,...,...,...
123,238,Xiaokuan Zhang,Assistant Professor,N/A,Xiaokuan Zhang's research interest include co...,N/A,PhD; Computer Science and Engineering; Ohio St...,https://cec.gmu.edu/profiles/xzhang46,xzhang46@gmu.edu,N/A,"System Security, Privacy, Side Channels, DeFi,...",side channels
124,240,Ziwei Zhu,Assistant Professor,N/A,Ziwei Zhu is broadly interested in data minin...,N/A,PhD; Computer Science; Texas A&M University\n,https://cec.gmu.edu/profiles/zzhu20,zzhu20@gmu.edu,N/A,"data mining, information retrieval, machine le...",responsible ai
124,240,Ziwei Zhu,Assistant Professor,N/A,Ziwei Zhu is broadly interested in data minin...,N/A,PhD; Computer Science; Texas A&M University\n,https://cec.gmu.edu/profiles/zzhu20,zzhu20@gmu.edu,N/A,"data mining, information retrieval, machine le...",data mining
124,240,Ziwei Zhu,Assistant Professor,N/A,Ziwei Zhu is broadly interested in data minin...,N/A,PhD; Computer Science; Texas A&M University\n,https://cec.gmu.edu/profiles/zzhu20,zzhu20@gmu.edu,N/A,"data mining, information retrieval, machine le...",machine learning


In [235]:
#Remove spaces from column names and replace with _
CEC_GS = CEC_GS.rename(columns={'Phone Number': 'Phone_Number', 'Research Interest': 'Research_Interest', 
                               'Profile Link': 'Profile_Link', 'Personal Link': 'Personal_Link',
                               'Google Scholar topics': 'Google_Scholar_topics'})

    ID            Name                                              Title  \
34  60  Rajesh Ganesan  Associate Professor, Systems Engineering and O...   
34  60  Rajesh Ganesan  Associate Professor, Systems Engineering and O...   
34  60  Rajesh Ganesan  Associate Professor, Systems Engineering and O...   
34  60  Rajesh Ganesan  Associate Professor, Systems Engineering and O...   
34  60  Rajesh Ganesan  Associate Professor, Systems Engineering and O...   
34  60  Rajesh Ganesan  Associate Professor, Systems Engineering and O...   
34  60  Rajesh Ganesan  Associate Professor, Systems Engineering and O...   

    Phone_Number                                          Biography  \
34  703-993-1693   Operations research deals with the applicatio...   
34  703-993-1693   Operations research deals with the applicatio...   
34  703-993-1693   Operations research deals with the applicatio...   
34  703-993-1693   Operations research deals with the applicatio...   
34  703-993-1693   Operation

In [236]:
#Savle files
common.to_csv('common_names.csv')
missing.to_csv('missing_names.csv')
common_and_missing.to_csv('common_and_missing.csv')
CEC_GS.to_csv('CEC_GS.csv')

In [237]:
#Take all areas and categories in current dictionary
vse['Topic_l'] = vse['Topic'].str.lower()
vse['Area_l'] = vse['Area'].str.lower()
vse['Category_l'] = vse['Category'].str.lower()
Area_unique = vse['Area_l'].unique()
Cat_unique = vse['Category_l'].unique()


In [238]:
#Create dataframe that holds all All_Reseach_Interests that exist within  areas of dictionary; Save to file
in_area = CEC_GS[CEC_GS['All_Research_Interest'].isin(Area_unique)].copy()
print(len(in_area))
area_ids = []
for i in in_area['All_Research_Interest']:
    
    area_ids.append(vse.loc[vse['Area'] == i, 'Area_ID'].unique()[0])

area_ids
#in_area['Area_ID'] = ""
in_area['Area_ID'] = list(area_ids)
in_area
in_area.to_csv('in_area.csv')

41


In [239]:
#Create dataframe that holds all All_Reseach_Interests that exist within  categoires of dictionary; Save to file
in_cat = CEC_GS[CEC_GS['All_Research_Interest'].isin(Cat_unique)].copy()
print(len(in_cat))
cat_ids = []
for i in in_cat['All_Research_Interest']:
    
    cat_ids.append(vse.loc[vse['Category'] == i, 'Category_ID'].unique()[0])


#in_area['Area_ID'] = ""
in_cat['Category_ID'] = list(cat_ids)
in_cat
in_cat.to_csv('in_cat.csv')

54


In [241]:
#Create dataframe that holds all All_Reseach_Interests that dont exist within  categoires or areas of dictionary
#Save to file
not_in_cat = CEC_GS[~CEC_GS['All_Research_Interest'].isin(Cat_unique)].copy()
not_in_area = CEC_GS[~CEC_GS['All_Research_Interest'].isin(Area_unique)].copy()
not_in_area
s1 = pd.merge(not_in_cat, not_in_area, how='inner', on=list(not_in_area.columns.values))
s1.to_csv('not_in_cat_or_area.csv')